## Segmenting and Clustering Neighbourhoods in Toronto City

I will connect to the wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and scrape the required data. Also used BeautifulSoup to obtain the data from table. Then parse into DataFrame. I will use Geodata by google(if applicable) or use the provided the csv file that contains latitude and longitude for the assinged postal code. 
I will use the Foursquare API to explore neighborhoods in Toronto City. The neighbourhood will be narrowed down by only include the neighbourhoods contains word "Toronto". The I will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. We will use the k-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in Toronto City and their emerging clusters.

Data sources : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and 
               https://cocl.us/Geospatial_data


In [ ]:
# The code was removed by Watson Studio for sharing.

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes 

import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

connect to the wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and scrape the required data.  Also used BeautifulSoup to obtain the data from table.  Then parse into DataFrame.




In [4]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Check the shape of the total initial data.  


In [5]:
df.shape

(287, 3)

The data contains 287 rows and 3 columns.  Check how many rows of Not assinged value in Borough column

In [6]:
df[df.Borough == 'Not assigned'].shape

(77, 3)

We want to only process the values that have an assinged Borough. So we will remove 77 'Not assinged' Borough rows.

In [7]:
# Remove all rows for 'Not assinged' Borough.   
df = df[df.Borough != 'Not assigned']
#df.drop(df.loc[df['Borough'] == 'Not assigned'].index, inplace=True)
df.shape

(210, 3)

Check how many multiple neighborhood exist in one postal code area.  For example, M9v and M8Y have 8 neighborhoods. 

In [8]:
df['Postcode'].value_counts()

M8Y    8
M9V    8
M5V    7
M9B    5
M4V    5
M8Z    5
M9R    4
M9C    4
M1V    4
M6M    4
M5R    3
M5J    3
M8V    3
M5T    3
M6L    3
M1K    3
M1T    3
M1E    3
M2J    3
M1C    3
M1P    3
M3H    3
M6K    3
M1M    3
M1L    3
M8X    3
M5H    3
M6A    2
M5K    2
M6R    2
      ..
M2N    1
M6B    1
M2P    1
M4H    1
M3B    1
M7Y    1
M4S    1
M1S    1
M5N    1
M4P    1
M4N    1
M7A    1
M1H    1
M6C    1
M4R    1
M1W    1
M4J    1
M3A    1
M9P    1
M4A    1
M3N    1
M6E    1
M4M    1
M2H    1
M7R    1
M1G    1
M4E    1
M3L    1
M9N    1
M5C    1
Name: Postcode, Length: 103, dtype: int64

We want to join all neighborhoods under one postal code into one cell separated by ", ". 

In [9]:
df_group =  pd.DataFrame(df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join))

After group by 'Postcode' and 'Borough', two columns became an multiindex.  We want to reset index so dataframe now back to 3 columns.

In [10]:
df_group.reset_index(inplace=True)
df_group.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now check for Not assinged Neighbourhood. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [11]:
df_group[df_group['Neighbourhood']=='Not assigned'].size

0

Check the shape of the processed data. 

In [12]:
df_group.shape


(103, 3)

Install geocoder package 

In [13]:
!pip install geocoder

Try to use geocoder in multiple different ways but it kept returning 'None'.  While loop is keep running for a while so we will use the provide the csv file for the course. The is located in https://cocl.us/Geospatial_data

In [14]:

import geocoder
lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [15]:
geocode_df = pd.read_csv('https://cocl.us/Geospatial_data')
geocode_df.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
geocode_df.shape

(103, 3)

Notice that csv file contains a column name "Postal Code".  The data from wikipedia has "Postcode". The columns are mismatching so we will rename the 'Postal Code' to 'Postcode'

In [17]:
geocode_df.rename(columns = {"Postal Code":"Postcode"}, inplace=True)
geocode_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now we have two data frames and those will be joined on Postcode. After joined, the new column has created as index.

In [18]:
toronto = df_group.join(geocode_df.set_index('Postcode'), on='Postcode')
toronto.head()



,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
#toronto.drop("index", axis=1, inplace=True)
toronto.rename(columns={"Postcode":"PostalCode"}, inplace=True)

In [20]:
toronto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


In [42]:

toronto[toronto['Neighbourhood'].str.contains('University of Toronto', regex=False)]


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
66,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049


In [22]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto['Borough'].unique()),
        toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [34]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [38]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
Define Foursquare Credentials and Version

In [39]:
CLIENT_ID = 'M45GUPMUJY2QUKJTVR5COOBBOGMWVTXCECKDOZN40MCXWQS5' # your Foursquare ID
CLIENT_SECRET = '20YMK3Y510PMJY015NIA0TWTI4CUUCJMRWZQ0ZDWDR55P1Z5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M45GUPMUJY2QUKJTVR5COOBBOGMWVTXCECKDOZN40MCXWQS5
CLIENT_SECRET:20YMK3Y510PMJY015NIA0TWTI4CUUCJMRWZQ0ZDWDR55P1Z5


We will explore the neighbourhood of University of Toronto.  The index of University of Toronto is 66. 

In [44]:
uofT = toronto[toronto['Neighbourhood'].str.contains('University of Toronto', regex=False)]
uofT

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
66,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049


In [66]:
neighbourhood_latitude = toronto.loc[66, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto.loc[66, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto.loc[66, 'Neighbourhood'] # neighborhood name
neighbourhood_pcode = toronto.loc[66, 'PostalCode'] # neighborhood name


print('Postal Code, Latitude and longitude values of {} are {}, {}, {}.'.format(neighbourhood_name, 
                                                                                neighbourhood_pcode, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Postal Code, Latitude and longitude values of Harbord, University of Toronto are M5S, 43.6626956, -79.4000493.


#### Now, let's get the top 100 venues that are in U of T within a radius of 500 meters



First, let's create the GET request URL. Name your URL url.

In [46]:
# type your answer here
LIMIT = 100
radius = 500
#url = "https://api.foursquare.com/v2/venues/explore?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=neighborhood_latitude,neighborhood_longitude&v=VERSION&limit=LIMIT"
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}\
&ll={},{}&v={}&radius={}&limit={}'.format(
CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, radius, LIMIT)


url

'https://api.foursquare.com/v2/venues/explore?client_id=M45GUPMUJY2QUKJTVR5COOBBOGMWVTXCECKDOZN40MCXWQS5&client_secret=20YMK3Y510PMJY015NIA0TWTI4CUUCJMRWZQ0ZDWDR55P1Z5&ll=43.6626956,-79.4000493&v=20180605&radius=500&limit=100'

Send the GET request and examine the results

In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e5dd173bae9a2001c6c5432'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'University of Toronto',
  'headerFullLocation': 'University of Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 37,
  'suggestedBounds': {'ne': {'lat': 43.6671956045, 'lng': -79.39384042790832},
   'sw': {'lat': 43.6581955955, 'lng': -79.4062581720917}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5362c366498e602fbe1db395',
       'name': 'Yasu',
       'location': {'address': '81 Harbord St.',
        'lat': 43.66283719650635,
        'lng': -79.40321739973975,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.662

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's clean the json and structure it into a pandas dataframe

In [49]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746
2,Rasa,Restaurant,43.662757,-79.403988
3,Piano Piano,Italian Restaurant,43.662949,-79.402898
4,Her Father's Cider Bar + Kitchen,Beer Bar,43.662448,-79.404703


In [50]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

37 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto(Neighbourhood contains Toronto)

In [51]:
toronto_data = toronto[toronto['Neighbourhood'].str.contains('Toronto', regex=False)]

In [74]:
toronto_data.shape

(7, 5)

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [75]:
def getNearbyVenues(pcodes, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for pcode, name, lat, lng in zip(pcodes, names, latitudes, longitudes):
        print(pcode + ' '+ name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            pcode,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode',
                             'Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



#### The code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*

In [76]:
toronto_venues = getNearbyVenues(pcodes = toronto_data['PostalCode'],
                                    names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



M3K CFB Toronto, Downsview East
M4J East Toronto
M4R North Toronto West
M5J Harbourfront East, Toronto Islands, Union Station
M5K Design Exchange, Toronto Dominion Centre
M5S Harbord, University of Toronto
M8V Humber Bay Shores, Mimico South, New Toronto


In [77]:
print(toronto_venues.shape)
toronto_venues.head()

(275, 8)


,PostalCode,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3K,"CFB Toronto, Downsview East",43.737473,-79.464763,Toronto Downsview Airport (YZD),43.738883,-79.470111,Airport
1,M3K,"CFB Toronto, Downsview East",43.737473,-79.464763,Ancaster Park,43.734706,-79.464777,Park
2,M3K,"CFB Toronto, Downsview East",43.737473,-79.464763,Fly By (Bombardier),43.737632,-79.469056,Snack Place
3,M3K,"CFB Toronto, Downsview East",43.737473,-79.464763,VAUGHAN GARAGE DOOR REPAIR,43.734877,-79.468584,Other Repair Shop
4,M4J,East Toronto,43.685347,-79.338106,The Path,43.683923,-79.335007,Park


Let's check how many venues were returned for each neighborhood

In [81]:
toronto_venues.groupby('PostalCode').count()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,,
M3K,4,4,4,4,4,4,4
M4J,3,3,3,3,3,3,3
M4R,20,20,20,20,20,20,20
M5J,100,100,100,100,100,100,100
M5K,100,100,100,100,100,100,100
M5S,37,37,37,37,37,37,37
M8V,11,11,11,11,11,11,11


#### Let's find out how many unique categories can be curated from all the returned venues

In [82]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 100 uniques categories.


## Analyze Each Neighborhood

In [83]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

In [84]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,American Restaurant,Aquarium,Art Gallery,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,...,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,"CFB Toronto, Downsview East",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
toronto_onehot.shape

(275, 101)

In [86]:
toronto_onehot.groupby('Neighbourhood').mean()

,Airport,American Restaurant,Aquarium,Art Gallery,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,...,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Yoga Studio
Neighbourhood,,,,,,,,,,,,,,,,,,,,,
"CFB Toronto, Downsview East",0.25,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000
"Design Exchange, Toronto Dominion Centre",0.00,0.03,0.00,0.01,0.040000,0.040000,0.00,0.01,0.020000,0.000000,...,0.010000,0.01,0.02,0.01,0.010000,0.01,0.01,0.000000,0.01,0.000000
East Toronto,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000
"Harbord, University of Toronto",0.00,0.00,0.00,0.00,0.054054,0.054054,0.00,0.00,0.027027,0.027027,...,0.027027,0.00,0.00,0.00,0.027027,0.00,0.00,0.027027,0.00,0.027027
"Harbourfront East, Toronto Islands, Union Station",0.00,0.00,0.05,0.01,0.020000,0.020000,0.02,0.01,0.010000,0.000000,...,0.010000,0.00,0.01,0.00,0.010000,0.01,0.01,0.000000,0.01,0.000000
"Humber Bay Shores, Mimico South, New Toronto",0.00,0.00,0.00,0.00,0.090909,0.000000,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000
North Toronto West,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.050000


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [87]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,American Restaurant,Aquarium,Art Gallery,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,...,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,"CFB Toronto, Downsview East",0.25,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000
1,"Design Exchange, Toronto Dominion Centre",0.00,0.03,0.00,0.01,0.040000,0.040000,0.00,0.01,0.020000,...,0.010000,0.01,0.02,0.01,0.010000,0.01,0.01,0.000000,0.01,0.000000
2,East Toronto,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000
3,"Harbord, University of Toronto",0.00,0.00,0.00,0.00,0.054054,0.054054,0.00,0.00,0.027027,...,0.027027,0.00,0.00,0.00,0.027027,0.00,0.00,0.027027,0.00,0.027027
4,"Harbourfront East, Toronto Islands, Union Station",0.00,0.00,0.05,0.01,0.020000,0.020000,0.02,0.01,0.010000,...,0.010000,0.00,0.01,0.00,0.010000,0.01,0.01,0.000000,0.01,0.000000
5,"Humber Bay Shores, Mimico South, New Toronto",0.00,0.00,0.00,0.00,0.090909,0.000000,0.00,0.00,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000
6,North Toronto West,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.050000


In [88]:
toronto_grouped.shape

(7, 101)

In [89]:
toronto_grouped[toronto_grouped['Neighbourhood'] == 'CFB Toronto, Downsview East'].T

,0
Neighbourhood,"CFB Toronto, Downsview East"
Airport,0.25
American Restaurant,0
Aquarium,0
Art Gallery,0
Bakery,0
Bar,0
Baseball Stadium,0
Basketball Stadium,0
Beer Bar,0


#### Let's print each neighborhood along with the top 5 most common venues

In [90]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

----CFB Toronto, Downsview East----
----Design Exchange, Toronto Dominion Centre----
----East Toronto----
----Harbord, University of Toronto----
----Harbourfront East, Toronto Islands, Union Station----
----Humber Bay Shores, Mimico South, New Toronto----
----North Toronto West----


In [91]:
print(temp.sort_values('freq', ascending=False).reset_index(drop=False).head(num_top_venues))

   index           venue  freq
0     18  Clothing Store  0.15
1     19     Coffee Shop  0.10
2    100     Yoga Studio  0.05
3     16            Café  0.05
4     66            Park  0.05


In [92]:
print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
print('\n')

            venue  freq
0  Clothing Store  0.15
1     Coffee Shop  0.10
2     Yoga Studio  0.05
3            Café  0.05
4            Park  0.05




#### Let's put that into a *pandas* dataframe
First, let's write a function to sort the venues in descending order.

In [94]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CFB Toronto, Downsview East",Airport,Park,Other Repair Shop,Snack Place,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Event Space
1,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Bar,Bakery,American Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
2,East Toronto,Convenience Store,Coffee Shop,Park,Yoga Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Event Space,Fast Food Restaurant
3,"Harbord, University of Toronto",Café,Bookstore,Italian Restaurant,Japanese Restaurant,Bakery,Bar,Restaurant,Pub,Sandwich Place,Coffee Shop
4,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Aquarium,Café,Hotel,Brewery,Italian Restaurant,Scenic Lookout,Fried Chicken Joint,Restaurant,Bakery
5,"Humber Bay Shores, Mimico South, New Toronto",Café,Restaurant,Pizza Place,Bakery,Pharmacy,Gym,Liquor Store,Fast Food Restaurant,Coffee Shop,Fried Chicken Joint
6,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Spa,Health & Beauty Service,Grocery Store,Mexican Restaurant,Fast Food Restaurant,Diner,Park


## Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters

In [98]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 0, 0, 4, 3], dtype=int32)

In [100]:
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
30,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763
40,M4J,East York,East Toronto,43.685347,-79.338106
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576


In [101]:
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CFB Toronto, Downsview East",Airport,Park,Other Repair Shop,Snack Place,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Event Space
1,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Bar,Bakery,American Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
2,East Toronto,Convenience Store,Coffee Shop,Park,Yoga Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Event Space,Fast Food Restaurant
3,"Harbord, University of Toronto",Café,Bookstore,Italian Restaurant,Japanese Restaurant,Bakery,Bar,Restaurant,Pub,Sandwich Place,Coffee Shop
4,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Aquarium,Café,Hotel,Brewery,Italian Restaurant,Scenic Lookout,Fried Chicken Joint,Restaurant,Bakery


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [103]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [104]:
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763,1,Airport,Park,Other Repair Shop,Snack Place,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Event Space
40,M4J,East York,East Toronto,43.685347,-79.338106,2,Convenience Store,Coffee Shop,Park,Yoga Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Event Space,Fast Food Restaurant
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Clothing Store,Coffee Shop,Yoga Studio,Spa,Health & Beauty Service,Grocery Store,Mexican Restaurant,Fast Food Restaurant,Diner,Park
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Café,Hotel,Brewery,Italian Restaurant,Scenic Lookout,Fried Chicken Joint,Restaurant,Bakery
60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,Bar,Bakery,American Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant


Finally, let's visualize the resulting clusters

In [105]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



## Examine Clusters


Cluster 1

In [107]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Brewery,Italian Restaurant,Scenic Lookout,Fried Chicken Joint,Restaurant,Bakery
60,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Bar,Bakery,American Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
66,Downtown Toronto,0,Café,Bookstore,Italian Restaurant,Japanese Restaurant,Bakery,Bar,Restaurant,Pub,Sandwich Place,Coffee Shop


Cluster 2

In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,North York,1,Airport,Park,Other Repair Shop,Snack Place,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Event Space


Cluster 3

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,East York,2,Convenience Store,Coffee Shop,Park,Yoga Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Event Space,Fast Food Restaurant


Cluster 4

In [111]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Central Toronto,3,Clothing Store,Coffee Shop,Yoga Studio,Spa,Health & Beauty Service,Grocery Store,Mexican Restaurant,Fast Food Restaurant,Diner,Park


Cluster 5

In [112]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,Etobicoke,4,Café,Restaurant,Pizza Place,Bakery,Pharmacy,Gym,Liquor Store,Fast Food Restaurant,Coffee Shop,Fried Chicken Joint
